## Project Ojective - Dog Breed Classification

We are provided with a training set and a test set of images of dogs. Each image has a filename that is its unique id. The dataset comprises 120 breeds of dogs. The objective of the project is to create a classifier capable of determining a dog's breed from a photo. We will use traditional CNN with data augmentation and Transfer Learning by VGG16 model with weights pre-trained on Imagenet for calssification of the dog breed.

### Load Dataset Files

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

import tensorflow as tf
import numpy as np
import pandas as pd

import time
from datetime import timedelta

import math
import os
import random

from sklearn.preprocessing import OneHotEncoder
import scipy.misc
from scipy.stats import itemfreq
from random import sample
import pickle

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout, Flatten, Reshape
from tensorflow.keras.layers import Convolution2D,MaxPooling2D,Dense, Activation, Dropout, Flatten, Reshape

from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split

# Image manipulation
import PIL.Image
from IPython.display import display

# Open a Zip File
from zipfile import ZipFile
from io import BytesIO

In [ ]:
import numpy as np
import pandas as pd
import keras
import tensorflow as tf
from keras.utils import np_utils
from sklearn import metrics
import matplotlib.pyplot as plt
%matplotlib inline
from zipfile import ZipFile

Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [ ]:
project_path = "dogImages/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels.

In [ ]:
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [ ]:
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [ ]:
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [ ]:
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

We now have 4 files - Train folder, test folder and labels.csv and sample_submission.csv 

### Read labels.csv file using pandas

In [ ]:
labels = pd.read_csv(project_path+'labels.csv.zip')

In [ ]:
labels.head(10)

In [ ]:
labels.shape

### Print the count of each category of Dogs given in the dataset



In [ ]:
breed_count = labels['breed'].value_counts()

In [ ]:
breed_count.head()

In [ ]:
breed_count.shape

### Get one-hot encodings of labels

In [ ]:
targets = pd.Series(labels['breed'])
one_hot = pd.get_dummies(targets, sparse=True)
one_hot_labels = np.asarray(one_hot)

In [ ]:
one_hot_labels

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [ ]:
img_rows = 128
img_col = 128 
img_channel = 1

from tqdm import tqdm
import cv2

In [ ]:
x_train = []
y_train = []

for id,breed in tqdm(labels.values):
  train_img = cv2.imread('./train/{}.jpg'.format(id),1)
  train_img_resize = cv2.resize(train_img, (img_rows,img_col))
  x_train.append(train_img_resize)
  y_train.append(breed)

In [ ]:
x_train[0].shape

In [ ]:
y_train[0]

In [ ]:
import matplotlib.pyplot as plt
image =x_train[0]

im2 = image.copy()
im2[:, :, 0] = image[:, :, 2]
im2[:, :, 2] = image[:, :, 0]
plt.imshow(im2)


In [ ]:


%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.figure(figsize=(12, 6))
for i in range(8):
    random_index = random.randint(0, 10221)
    plt.subplot(2, 4, i+1)
    plt.imshow(x_train[random_index][:,:,::-1])
    plt.title(y_train[random_index])
    plt.axis('off')

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [ ]:
y_train = pd.get_dummies(y_train, sparse=True)


In [ ]:
x_train = np.array(x_train)
y_train = np.array(y_train)

In [ ]:
y_train.shape

In [ ]:
x_train.shape

In [ ]:
y_train.dtype

In [ ]:
x_train = x_train.astype('float32')
y_train = y_train.astype('float32')
x_train /= 255
#y_train /= 255

In [ ]:
y_train[0]

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [ ]:
x_train2, x_val, y_train2, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=2)
print (len(x_train2))
print (len(x_val))

In [ ]:
x_train2[0].shape

In [ ]:
x_train2 = x_train2.reshape(x_train2.shape[0],128,128,3)

In [ ]:
x_val = x_val.reshape(x_val.shape[0],128,128,3)

In [ ]:
print (x_train2.shape)
print (x_val.shape)

In [ ]:
print(y_train2.shape)
print(y_val.shape)

In [ ]:
print(y_val[0])

In [ ]:
import matplotlib.pyplot as plt
image =x_train2[0]

im2 = image.copy()
im2[:, :, 0] = image[:, :, 2]
im2[:, :, 2] = image[:, :, 0]
plt.imshow(im2)


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [ ]:
import pandas as pd
sample_submission = pd.read_csv(project_path+'sample_submission.csv.zip')

In [ ]:
sample_submission.head()

In [ ]:
test_img=sample_submission['id']


In [ ]:
test_img.shape

In [ ]:
test_img.head()

Run the below code to load the test image files in x_test_feature

In [ ]:
img_rows = 128
img_col = 128 
img_channel = 1

x_test_feature = []

i = 0 # initialisation
for f in tqdm(test_img.values): # f for format ,jpg
    img = cv2.imread('./test/{}.jpg'.format(f), 1)
    img_resize = cv2.resize(img, (img_rows, img_col)) 
    x_test_feature.append(img_resize)

In [ ]:
x_test_feature[0].shape

Normalize the test data and convert it into 4 dimensions

In [ ]:
import numpy as np
x_test = np.array(x_test_feature)

In [ ]:
x_test = x_test.astype('float32')
x_test /= 255

In [ ]:
x_test.shape

In [ ]:
x_test = x_test.reshape(x_test.shape[0],128,128,3)

In [ ]:
print (x_test.shape)

In [ ]:
image =x_test[0]

im2 = image.copy()
im2[:, :, 0] = image[:, :, 2]
im2[:, :, 2] = image[:, :, 0]
plt.imshow(im2)


In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

plt.figure(figsize=(12, 6))
for i in range(8):
    random_index = random.randint(0, 10221)
    plt.subplot(2, 4, i+1)
    plt.imshow(x_test[random_index][:,:,::-1])
    plt.title(test_img[random_index])
    plt.axis('off')

Note: x_test and corresponding label IDs will not be used instead the train and val data will be used for model building and testing.

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [ ]:
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import BatchNormalization
   
# 1st CNN Layer

model = tf.keras.Sequential()
model.add(Convolution2D(32,5, 5, input_shape=(128, 128, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))

# 2nd CNN Layer

model.add(Convolution2D(32,3, 3, input_shape=(128, 128, 3)))
model.add(BatchNormalization(axis=3, scale=False))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(4, 4), strides=(4, 4), padding='same'))
model.add(Dropout(0.2))

# Fully Connected Layer
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu'))

# Prediction Layer
model.add(Dense(120))
model.add(Activation('softmax'))

In [ ]:
# Loss and Optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Store Training Results
early_stopping = EarlyStopping(monitor='val_loss', patience=7, verbose=1, mode='auto')
callback_list = [early_stopping]

In [ ]:
# Train the model
model.fit(x_train2, y_train2, batch_size=10, nb_epoch=10, validation_data=(x_val, y_val))

### Use batch_size = 128 and epochs = 50 and execute the model

In [ ]:
TRAIN = False
BATCH_SIZE = 128
EPOCHS = 50

In [ ]:
history = model.fit(x_train2, y_train2,epochs=EPOCHS, batch_size=BATCH_SIZE, verbose=1,validation_data=(x_val,y_val))

Poor Model Accuracy even after 50 epochs..!

### Use Data Augmentation in the above model to see if the accuracy improves


In [ ]:
from keras.preprocessing.image import ImageDataGenerator
# This will do preprocessing and realtime data augmentation:
datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    rotation_range=50,  # randomly rotate images in the range (degrees, 0 to 180)
    width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
    height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
    horizontal_flip=True,  # randomly flip images
    vertical_flip=True)  # randomly flip images

datagen.fit(x_train2)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [ ]:
from matplotlib import pyplot as plt
train_generator = datagen.flow(x_train2[:1], batch_size=1)
for i in range(1, 6):
    plt.subplot(1,5,i)
    plt.axis("off")
    plt.imshow(train_generator.next().squeeze(), cmap='Blues')
    plt.plot()
plt.show()

In [ ]:
from matplotlib import pyplot as plt
val_generator = datagen.flow(x_val[0:1], batch_size=1)
for i in range(1, 6):
    plt.subplot(1,5,i)
    plt.axis("off")
    plt.imshow(val_generator.next().squeeze(), cmap='gray')
    plt.plot()
plt.show()

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [ ]:
history =model.fit(datagen.flow(x_train2, y_train2,batch_size=1), steps_per_epoch=x_train2.shape[0], epochs=10, validation_data=(x_val, y_val), callbacks=callback_list)

Model accuracy is still very poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=(project_path+'vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'),
                 include_top=False, pooling='avg')

Print the summary of the base_model

In [ ]:
base_model.summary()

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.models import *
from tensorflow.keras.layers import BatchNormalization, GlobalAveragePooling3D


x = base_model.output
x = Flatten()(x)
x = Dropout(0.2)(x)
# let's add two fully-connected layer
x = Dense(1024, activation='relu')(x)
x = Dense(256, activation='relu')(x)

# and a softmax layer for 120 classes
predictions = Dense(120, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)


In [ ]:
# summarize
model.summary()

### Make all the layers in the base_model (VGG16) to be non-trainable

In [ ]:
for layer in base_model.layers:
  #if('conv' in layer.name): #prefix detection to freeze layers which does not have dense
    #Freezing a layer
    layer.trainable = False

#Module to print colourful statements
from termcolor import colored

#Check which layers have been frozen 
for layer in base_model.layers:
  print (colored(layer.name, 'blue'))
  print (colored(layer.trainable, 'red'))

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

In [ ]:
# Loss and Optimizer
from tensorflow.keras.optimizers import Adam
from keras.losses import categorical_crossentropy

model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

Try to get training and validation accuracy to be more than 90%

In [ ]:
history = model.fit(x_train2, y_train2,epochs=10, batch_size=128, verbose=1,validation_data=(x_val,y_val))

In [ ]:
def plot_acc_loss(history):
    fig = plt.figure(figsize=(10,5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
 
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'test'], loc='upper right')
    plt.show()
 
plot_acc_loss(history)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
predictions = np.argmax(model.predict(x_val), axis=1)

In [ ]:
predictions

In [ ]:
y_val[1]

In [ ]:
y_val_label=np.argmax(y_val, axis=1)

In [ ]:
y_val_label

In [ ]:
print("\nAccuracy on Test Data: ", accuracy_score(y_val_label, predictions))
print("\nNumber of correctly identified images: ",
      accuracy_score(y_val_label, predictions, normalize=False),"\n")
confusion_matrix(y_val_label, predictions)

Leveraging VGG16 Network to see if the accuracy score improves ?

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
# Creating a checkpointer 
checkpointer = ModelCheckpoint(filepath=(project_path+'vgg16_weights_full_model.h5'), 
                               verbose=1,save_best_only=True)

In [ ]:
# load the VGG16 network 
print("[INFO loading network...")
model_vgg = VGG16(weights="imagenet", include_top=False, input_shape=x_train.shape[1:])
model_vgg.summary()

In [ ]:
from tensorflow.keras.layers import Dropout, Flatten, GlobalAveragePooling2D
model_transfer_full = Sequential()
model_transfer_full.add(model_vgg)
model_transfer_full.add(GlobalAveragePooling2D())
model_transfer_full.add(Dropout(0.2))
model_transfer_full.add(Dense(1024, activation='relu'))
model_transfer_full.add(Dense(256, activation='relu'))
model_transfer_full.add(Dense(120, activation='softmax'))
model_transfer_full.summary()

In [ ]:
opt = Adam(lr=0.00001)
model_transfer_full.compile(loss='categorical_crossentropy', optimizer=opt,metrics=['accuracy'])
history = model_transfer_full.fit(x_train2, y_train2, batch_size=32, epochs=30,
          validation_data=(x_val, y_val),callbacks=[checkpointer],verbose=1, shuffle=True)

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
predictions = np.argmax(model_transfer_full.predict(x_val), axis=1)
y_val_label=np.argmax(y_val, axis=1)

In [ ]:
print("\nAccuracy on Test Data: ", accuracy_score(y_val_label, predictions))
print("\nNumber of correctly identified images: ",
      accuracy_score(y_val_label, predictions, normalize=False),"\n")
confusion_matrix(y_val_label, predictions)

Due to memory limitations, a small subset of the data (256/10221) is taken and operated on which leads to low accuracy levels (1%). On using transfer learning with VGG we obtain an accuracy level of 2-3 %.